In [20]:
# Install necessary packages
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [21]:
# Import dependencies
import openmeteo_requests
import requests_cache
import pandas as pd
import json
import requests
import os

from retry_requests import retry
from dotenv import load_dotenv, find_dotenv


# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [22]:
# Load EIA Api Key
load_dotenv(find_dotenv("local_keys.env"))
eia_key = os.getenv("eia")

# Setup EIA Energy API URL
eia_base_url = f"https://api.eia.gov/v2/electricity/electric-power-operational-data/data/?api_key={eia_key}"

In [25]:
# Setup Open-Meteo base request
open_meteo_base_url = "https://archive-api.open-meteo.com/v1/archive"

open_meteo_base_params = {
	"start_date": "2019-01-01",
	"end_date": "2023-12-31",
	"daily": ["temperature_2m_mean", "precipitation_sum"],
	"temperature_unit": "fahrenheit",
	"wind_speed_unit": "mph",
	"precipitation_unit": "inch"
}


In [26]:
# Get Honolulu Data
honoalulu_params = {
    **open_meteo_base_params,
    "latitude": 21.3069,  
    "longitude": -157.8583
}

responses = openmeteo.weather_api(open_meteo_base_url, params=honoalulu_params)
honolulu_response = response = responses[0]

In [27]:
# Process Honalulu Data
daily = honolulu_response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["Mean Temperature (f)"] = daily_temperature_2m_mean
daily_data["Total Precipitation (in)"] = daily_precipitation_sum



,date,Mean Temperature (f),Total Precipitation (in)
0,2019-01-01 00:00:00+00:00,75.174347,0.003937
1,2019-01-02 00:00:00+00:00,74.825607,0.000000
2,2019-01-03 00:00:00+00:00,74.308105,0.011811
3,2019-01-04 00:00:00+00:00,74.338097,0.000000
4,2019-01-05 00:00:00+00:00,74.004356,0.000000


In [39]:
# Remove NA and Normailze date format
honolulu_daily_df = pd.DataFrame(data = daily_data).dropna(how='any' )
honolulu_daily_df['date'] = honolulu_daily_dataframe['date'].dt.tz_localize(None)

honolulu_daily_df.head()

,date,Mean Temperature (f),Total Precipitation (in)
0,2019-01-01,75.174347,0.003937
1,2019-01-02,74.825607,0.000000
2,2019-01-03,74.308105,0.011811
3,2019-01-04,74.338097,0.000000
4,2019-01-05,74.004356,0.000000


In [40]:
# Convert daily data to monthly 
honolulu_daily_df.set_index('date', inplace=True)
honolulu_monthly_df = honolulu_daily_df.resample('M').mean()  # You can also use sum(), max(), min(), etc.

honolulu_monthly_df.reset_index(inplace=True)
honolulu_monthly_df['date'] = honolulu_monthly_df['date'].dt.strftime('%m/%Y')

honolulu_monthly_df

C:\Users\HollidayDDS\AppData\Local\Temp\ipykernel_31488\2199984611.py:3: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  honolulu_monthly_df = honolulu_daily_df.resample('M').mean()  # You can also use sum(), max(), min(), etc.


,date,Mean Temperature (f),Total Precipitation (in)
0,01/2019,72.986885,0.031496
1,02/2019,69.448105,0.079021
2,03/2019,72.239670,0.009779
3,04/2019,75.143097,0.028609
4,05/2019,76.997574,0.068580
5,06/2019,79.391228,0.137270
6,07/2019,80.425201,0.025146
7,08/2019,81.786568,0.030861
8,09/2019,81.418472,0.047900
9,10/2019,79.828346,0.032893
